In [ ]:
import pandas as pd
X_test = pd.read_csv('/content/drive/MyDrive/iot_magazine2020/x_test_resampled.csv', header=None)
X_test=X_test.values

In [ ]:
len(X_test)

2000

In [ ]:
#!pip install keras

In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
y_test = pd.read_csv('/content/drive/MyDrive/iot_magazine2020/y_test_resampled.csv', header=None)
y_test= y_test.values.reshape((len(y_test.values),1))
y_test = to_categorical(y_test)
y_test.shape

(2000, 4)

In [ ]:
import tensorflow as tf
import collections

x_data_fed = []
y_data_fed = []
x_data_fed2 = []
y_data_fed2 = []
x_data_fed3 = []
y_data_fed3 = []
x_data_fed4 = []
y_data_fed4 = []

data = []
data2 = []
data3= []
data4 = []


counter = 0
for i in range(25):
  x_data_fed.append([i.reshape(-1,21,1)[0] for i in X_test[counter:counter+10]])
  y_data_fed.append([i.reshape(-1,4)[0] for i in y_test[counter:counter+10]])
  counter=counter+10
for i in range(25):
  data.append(tf.data.Dataset.from_tensors(collections.OrderedDict(x=x_data_fed[i]  , y=y_data_fed[i] )))

print(counter)

for i in range(25):
  x_data_fed2.append([i.reshape(-1,21,1)[0] for i in X_test[counter:counter+10]])
  y_data_fed2.append([i.reshape(-1,4)[0] for i in y_test[counter:counter+10]])
  counter=counter+10
for i in range(25):
  data2.append(tf.data.Dataset.from_tensors(collections.OrderedDict(x=x_data_fed2[i]  , y=y_data_fed2[i] )))

print(counter)

for i in range(25):
  x_data_fed3.append([i.reshape(-1,21,1)[0] for i in X_test[counter:counter+10]])
  y_data_fed3.append([i.reshape(-1,4)[0] for i in y_test[counter:counter+10]])
  counter=counter+10
for i in range(25):
  data3.append(tf.data.Dataset.from_tensors(collections.OrderedDict(x=x_data_fed3[i]  , y=y_data_fed3[i] )))

print(counter)

for i in range(25):
  x_data_fed4.append([i.reshape(-1,21,1)[0] for i in X_test[counter:counter+10]])
  y_data_fed4.append([i.reshape(-1,4)[0] for i in y_test[counter:counter+10]])
  counter=counter+10

for i in range(25):
  data4.append(tf.data.Dataset.from_tensors(collections.OrderedDict(x=x_data_fed4[i]  , y=y_data_fed4[i] )))

print(counter)




print(counter)
print(data[0].element_spec)
print(data2[0].element_spec)
print(data3[0].element_spec)
print(data4[0].element_spec)



250
500
750
1000
1000
OrderedDict([('x', TensorSpec(shape=(10, 21, 1), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(10, 4), dtype=tf.float32, name=None))])
OrderedDict([('x', TensorSpec(shape=(10, 21, 1), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(10, 4), dtype=tf.float32, name=None))])
OrderedDict([('x', TensorSpec(shape=(10, 21, 1), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(10, 4), dtype=tf.float32, name=None))])
OrderedDict([('x', TensorSpec(shape=(10, 21, 1), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(10, 4), dtype=tf.float32, name=None))])


In [ ]:
!pip install tensorflow_federated_nightly
!pip install nest_asyncio Set up an Image Classification Model 
!pip install tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

import numpy as np
import tensorflow_federated as tff
import pickle


     |████████████████████████████████| 675kB 4.3MB/s 
     |████████████████████████████████| 225kB 29.7MB/s 
     |████████████████████████████████| 174kB 51.4MB/s 
     |████████████████████████████████| 456.4MB 33kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 4.2MB 27.1MB/s 
     |████████████████████████████████| 471kB 45.3MB/s 
     |████████████████████████████████| 12.8MB 151kB/s 
     |████████████████████████████████| 5.5MB 33.2MB/s 
     |████████████████████████████████| 1.3MB 26.5MB/s 
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement keras-nightly~=2.5.0.dev, but you'll have keras-nightly 2.7.0.dev2021070800 which is incompatible.
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement cachetools>=4.2.1, but you'll have cachetools 3.1.1 

ERROR: Could not find a version that satisfies the requirement Set (from versions: none)
ERROR: No matching distribution found for Set


In [ ]:
from tensorflow.keras.models import load_model



loaded_model = load_model('/content/drive/MyDrive/iot_magazine2020/CTG_resampled.h5')


opt=keras.optimizers.Adam()
loaded_model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = tf.keras.models.clone_model(loaded_model)
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=data[0].element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam())
  
state = iterative_process.initialize()

state = tff.learning.state_with_new_model_weights(
    state,
    trainable_weights=[v.numpy() for v in loaded_model.trainable_weights],
    non_trainable_weights=[
        v.numpy() for v in loaded_model.non_trainable_weights
    ])


Instructions for updating:
Colocations handled automatically by placer.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('update_state_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('update_state_fn',), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [ ]:
loaded_model2 = load_model('/content/drive/MyDrive/iot_magazine2020/CTG_resampled.h5', compile=False)


In [ ]:
from sklearn.metrics import classification_report

y_test2 = pd.read_csv('/content/drive/MyDrive/iot_magazine20#from RegscorePy import *

print('Round {r}'.format(r=1))20/y_test_resampled.csv', header=None)
y_test2= y_test2.values.reshape((len(y_test2.values),1))
x_test2 = pd.read_csv('/content/drive/MyDrive/iot_magazine2020/x_test_resampled.csv', header=None)
x_test2= x_test2.values.reshape((len(x_test2),21,1))
print(x_test2.shape)

def keras_evaluate(state, round_num):
  # Take our global model weights and push them back into a Keras model to
  # use its standard `.evaluate()` method.
  keras_model = loaded_model2
  keras_model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.CategoricalAccuracy()])
  state.model.assign_weights_to(keras_model)
  loss, accuracy = keras_model.evaluate(x_test2[counter:],y_test[counter:] )
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))
  predz = keras_model.predict(x_test2[counter:]).argmax(axis=-1)
  print(classification_report(y_test2[counter:],predz) )

(2000, 21, 1)


In [ ]:
#from RegscorePy import *

print('Round {r}'.format(r=1))
state, metrics = iterative_process.next(state, data)
train_metrics = metrics['train']
print(" train mterics: ", train_metrics)
keras_evaluate(state, 1)

print('Round {r}'.format(r=2))
state, metrics = iterative_process.next(state, data2)
train_metrics = metrics['train']
print("mterics: ", train_metrics)
keras_evaluate(state, 2)



print('Round {r}'.format(r=3))
state, metrics = iterative_process.next(state, data3)
train_metrics = metrics['train']
print("mterics: ", train_metrics)
keras_evaluate(state, 3)



print('Round {r}'.format(r=4))
state, metrics = iterative_process.next(state, data4)
train_metrics = metrics['train']
print("mterics: ", train_metrics)
keras_evaluate(state, 4)

Round 1
 train mterics:  OrderedDict([('categorical_accuracy', 0.624), ('loss', 1.18585)])
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4870: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  [0. 0. 0.]


32/32 [==============================] - 1s 11ms/step - loss: 0.0805 - categorical_accuracy: 0.9730
	Eval: loss=0.081, accuracy=0.973
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
              precision    recall  f1-score   support

           1       1.00      0.92      0.96       337
           2       0.92      0.99      0.96       336
           3       0.99      0.99      0.99       327

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000

Round 4
mterics:  OrderedDict([('categorical_accuracy', 0.976), ('loss', 0.06058333)])
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requ

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
32/32 [==============================] - 1s 12ms/step - loss: 0.1020 - categorical_accuracy: 0.9670
	Eval: loss=0.102, accuracy=0.967
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
              precision    recall  f1-score   support

           1       1.00      0.92      0.96       337
           2       0.92      0.99      0.95       336
           3       0.98      0.99      0.99       327

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000

